In [3]:
import pandas as pd
import os

pbp_path = "data/hockey_scraper_data/pbp"

In [4]:
name_2007_2007 = "nhl_pbp20072008.csv"

df = pd.read_csv(os.path.join(pbp_path, name_2007_2007))
display(df)

,Game_Id,Date,Period,Event,Description,Time_Elapsed,Seconds_Elapsed,Strength,Ev_Zone,Type,...,Away_Score,Home_Score,Away_Goalie,Away_Goalie_Id,Home_Goalie,Home_Goalie_Id,xC,yC,Home_Coach,Away_Coach
0,20001,2007-09-29,1,STOP,RINK REPAIR,0:00,0.0,5x5,NaN,NaN,...,0,0,ILYA BRYZGALOV,8468524.0,JONATHAN BERNIER,8473541.0,NaN,NaN,MARC CRAWFORD,RANDY CARLYLE
1,20001,2007-09-29,1,PSTR,Period Start- Local time: 5:31 BST,0:00,0.0,5x5,NaN,NaN,...,0,0,ILYA BRYZGALOV,8468524.0,JONATHAN BERNIER,8473541.0,NaN,NaN,MARC CRAWFORD,RANDY CARLYLE
2,20001,2007-09-29,1,FAC,ANA won Neu. Zone - ANA #19 MCDONALD vs L.A #1...,0:00,0.0,5x5,Neu,NaN,...,0,0,ILYA BRYZGALOV,8468524.0,JONATHAN BERNIER,8473541.0,0.0,0.0,MARC CRAWFORD,RANDY CARLYLE
3,20001,2007-09-29,1,SHOT,"L.A ONGOAL - 11 KOPITAR, Wrist, Off. Zone, 22 ft.",0:17,17.0,5x5,Off,WRIST SHOT,...,0,0,ILYA BRYZGALOV,8468524.0,JONATHAN BERNIER,8473541.0,-70.0,-11.0,MARC CRAWFORD,RANDY CARLYLE
4,20001,2007-09-29,1,HIT,"ANA #10 PERRY HIT L.A #4 BLAKE, Off. Zone",0:25,25.0,5x5,Off,NaN,...,0,0,ILYA BRYZGALOV,8468524.0,JONATHAN BERNIER,8473541.0,NaN,NaN,MARC CRAWFORD,RANDY CARLYLE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
382953,30416,2008-06-04,3,STOP,GOALIE STOPPED,19:26,1166.0,5x5,NaN,NaN,...,3,2,CHRIS OSGOOD,8458568.0,NaN,NaN,NaN,NaN,MICHEL THERRIEN,MIKE BABCOCK
382954,30416,2008-06-04,3,FAC,DET won Def. Zone - DET #33 DRAPER vs PIT #87 ...,19:26,1166.0,5x5,Def,NaN,...,3,2,CHRIS OSGOOD,8458568.0,NaN,NaN,69.0,-23.0,MICHEL THERRIEN,MIKE BABCOCK
382955,30416,2008-06-04,3,SHOT,"PIT ONGOAL - 87 CROSBY, Backhand, Off. Zone, 3...",19:59,1199.0,5x5,Off,BACKHAND,...,3,2,CHRIS OSGOOD,8458568.0,NaN,NaN,62.0,14.0,MICHEL THERRIEN,MIKE BABCOCK
382956,30416,2008-06-04,3,PEND,Period End- Local time: 10:42 EDT,20:00,1200.0,5x5,NaN,NaN,...,3,2,CHRIS OSGOOD,8458568.0,NaN,NaN,NaN,NaN,MICHEL THERRIEN,MIKE BABCOCK


# get all of the years together

In [7]:
big_df = []

# iterate over every filename in the pbp_path directory
for filename in os.listdir(pbp_path):
    # if the file is a csv file
    if filename.endswith(".csv"):
        print(filename, " "*20, end="\r")
        # read the csv file into a pandas dataframe
        df = pd.read_csv(os.path.join(pbp_path, filename))
        
        # append the dataframe to the big_df
        big_df.append(df)
        
        
# concatenate all the dataframes in big_df into a single dataframe
big_df = pd.concat(big_df, ignore_index=True)
display(big_df)

,Game_Id,Date,Period,Event,Description,Time_Elapsed,Seconds_Elapsed,Strength,Ev_Zone,Type,...,Home_Score,Away_Goalie,Away_Goalie_Id,Home_Goalie,Home_Goalie_Id,xC,yC,Home_Coach,Away_Coach,Unnamed: 0
0,20001,2018-10-03,1,PSTR,Period Start- Local time: 7:16 EDT,0:00,0.0,5x5,NaN,NaN,...,0,CAREY PRICE,8471679.0,FREDERIK ANDERSEN,8475883.0,NaN,NaN,MIKE BABCOCK,CLAUDE JULIEN,NaN
1,20001,2018-10-03,1,FAC,MTL won Neu. Zone - MTL #13 DOMI vs TOR #34 MA...,0:00,0.0,5x5,Neu,NaN,...,0,CAREY PRICE,8471679.0,FREDERIK ANDERSEN,8475883.0,0.0,0.0,MIKE BABCOCK,CLAUDE JULIEN,NaN
2,20001,2018-10-03,1,SHOT,"MTL ONGOAL - #62 LEHKONEN, Backhand, Off. Zone...",0:29,29.0,5x5,Off,BACKHAND,...,0,CAREY PRICE,8471679.0,FREDERIK ANDERSEN,8475883.0,78.0,-19.0,MIKE BABCOCK,CLAUDE JULIEN,NaN
3,20001,2018-10-03,1,SHOT,"TOR ONGOAL - #44 RIELLY, Snap, Off. Zone, 52 ft.",0:49,49.0,5x5,Off,SNAP SHOT,...,0,CAREY PRICE,8471679.0,FREDERIK ANDERSEN,8475883.0,-37.0,-10.0,MIKE BABCOCK,CLAUDE JULIEN,NaN
4,20001,2018-10-03,1,SHOT,"MTL ONGOAL - #11 GALLAGHER, Wrist, Off. Zone, ...",1:00,60.0,5x5,Off,WRIST SHOT,...,0,CAREY PRICE,8471679.0,FREDERIK ANDERSEN,8475883.0,47.0,-23.0,MIKE BABCOCK,CLAUDE JULIEN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5838147,30417,2011-06-15,3,STOP,PUCK IN BENCHES,19:29,1169.0,5x5,NaN,NaN,...,0,TIM THOMAS,8460703.0,ROBERTO LUONGO,8466141.0,NaN,NaN,ALAIN VIGNEAULT,CLAUDE JULIEN,NaN
5838148,30417,2011-06-15,3,FAC,VAN won Neu. Zone - BOS #11 CAMPBELL vs VAN #2...,19:29,1169.0,5x5,Neu,NaN,...,0,TIM THOMAS,8460703.0,ROBERTO LUONGO,8466141.0,-20.0,-22.0,ALAIN VIGNEAULT,CLAUDE JULIEN,NaN
5838149,30417,2011-06-15,3,MISS,"VAN #6 SALO, Slap, Wide of Net, Neu. Zone, 100...",19:46,1186.0,5x5,Neu,SLAP SHOT,...,0,TIM THOMAS,8460703.0,ROBERTO LUONGO,8466141.0,-4.0,-36.0,ALAIN VIGNEAULT,CLAUDE JULIEN,NaN
5838150,30417,2011-06-15,3,PEND,Period End- Local time: 7:45 PDT,20:00,1200.0,5x5,NaN,NaN,...,0,TIM THOMAS,8460703.0,ROBERTO LUONGO,8466141.0,NaN,NaN,ALAIN VIGNEAULT,CLAUDE JULIEN,NaN


KeyboardInterrupt: 

In [10]:
big_df.to_feather("data/hockey_scraper_data/pbp/combined_pbp.feather")